In [1]:
import sys 
import ee
import geemap as ui

try:
    ee.Initialize()
    print('!El paquete de google earth se inicializo correctamente')
except ee.EEException  as e:
    print('!El paquete de google Earth no se pudo inicializar')
except:
    print("Unexpected error: ", sys.exc_info()[0])

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


!El paquete de google earth se inicializo correctamente


In [2]:
# Importar la región de interés desde el asset
Mexico = ee.FeatureCollection('projects/ee-zsj2031126019/assets/Mexico')

# Definir fechas de consulta
fechainicio = '2023-03-20'
fechafin = '2023-09-22'

# Consultar la colección de imágenes Landsat 8
img = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
           .filterBounds(Mexico) \
           .filterDate(fechainicio, fechafin) \
           .filterMetadata('CLOUD_COVER', 'Less_Than', 10)

# Crear un mosaico RGB usando bandas Landsat y recortar a México
imgMosaico = img.select(['SR_B4', 'SR_B3', 'SR_B2']).mosaic().clip(Mexico.geometry()) \
                .multiply(0.0000275).add(-0.2).multiply(255).toUint8()  # Normalizar a 8 bits

# Exportar la imagen como un archivo GeoTIFF con reducción de calidad
task = ee.batch.Export.image.toDrive(
    image=imgMosaico,
    description='Mosaico_Landsat8_Mexico',
    folder='Mosaico_Landsat',  # Carpeta en tu Google Drive
    fileNamePrefix='mosaico_landsat',  # Prefijo del archivo
    region=Mexico.geometry().bounds().getInfo()['coordinates'],  # Región a exportar
    scale=60,  # Resolución ajustada a 100 metros/píxel
    crs='EPSG:4326',
    maxPixels=1e13  # Ajuste de píxeles máximos para evitar errores
)

# Iniciar la tarea de exportación
task.start()

print('La tarea de exportación ha comenzado...')


La tarea de exportación ha comenzado...
